In [1]:
from torch_geometric.nn import GIN
import argparse
from matplotlib import pyplot as plt
import numpy as np
import torch
import sys, os

from torch_geometric.data import Data as Data_g
from torch_geometric.data import DataLoader as DataLoader_G
from torch.utils.data import DataLoader, TensorDataset


# import path
sys.path.append("../")
from filepath import ABSOLUTE_PATH

sys.path.append(ABSOLUTE_PATH)
from src.train.heatpipe_transformer import load_data, SAN
from src.train.heatpipe import renormalize
from src.train.train import Trainer
from src.utils.utils import (
    create_res,
    set_seed,
    get_time,
    save_config_from_args,
    get_parameter_net,
    find_max_min,
    relative_error,
)
import time

d:\Anaconda\envs\mpd\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\vscode/research/multphysics_simulation\src\model\diffusion.py:478: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


In [ ]:
device = "cuda"
num_layers_NF, hidden_dim = 4, 512
loss = np.load(
    "../../results/heatpipe/surrogate/transformer"
    + "_"
    + str(hidden_dim)
    + "_1024_"
    + str(num_layers_NF)
    + "/record.py"
)
loss

In [ ]:
model = SAN(
    input_dim=4,
    output_dim=2412,
    hidden_dim=hidden_dim,
    k=32,
    m=8,
    num_heads_PE=4,
    dim_feedforward_PE=64,
    dropout_PE=0.1,
    num_layers_PE=1,
    num_heads_NF=8,
    dim_feedforward_NF=1024,
    dropout_NF=0.1,
    num_layers_NF=num_layers_NF,
)
model.load_state_dict(
    torch.load(
        "../../results/heatpipe/surrogate/transformer"
        + "_"
        + str(hidden_dim)
        + "_1024_"
        + str(num_layers_NF)
        + "/model.pt"
    )["model"]
)

In [ ]:
_, test_loader = load_data(ABSOLUTE_PATH, 200, 800, device)

In [ ]:
for batch in test_loader:
    x, pe, y = batch
    y_p = renormalize(model(x, pe))
y_t = renormalize(y)  # .reshape(-1, 804, 3)

In [ ]:
relative_error(y_t[..., 0], y_p[..., 0]), relative_error(y_t[..., 1:], y_p[..., 1:])

In [20]:
from torch_geometric.utils import get_laplacian, degree, add_self_loops

m = 8
x_val = (
    torch.tensor(np.load(ABSOLUTE_PATH + "/data/heatpipe/x_val_element_as_one_node.npy"))
    .to(device)
    .float()
    .unsqueeze(0)
)
y_val_t = (
    torch.tensor(np.load(ABSOLUTE_PATH + "/data/heatpipe/y_val_element_as_one_node.npy"))
    .to(device)
    .float()
    .unsqueeze(0)
)

# ? Now all data share one graph
edge_index = torch.tensor(
    np.load(ABSOLUTE_PATH + "/data/heatpipe/adj_val_element_as_one_node.npy").transpose(1, 0), dtype=torch.long
).to(device)
edge_index, edge_weight = get_laplacian(edge_index)
adjacency_matrix = torch.sparse_coo_tensor(edge_index, edge_weight)
deg = torch.bincount(edge_index[0], minlength=64)
deg_matrix = torch.diag(deg)
laplacian_matrix = deg_matrix - adjacency_matrix

eigvals, eigvecs = torch.linalg.eigh(laplacian_matrix)  # low to high
top_eigvals = eigvals[-m:]  # top m
top_eigvecs = eigvecs[:, -m:]
eigvals_ep = top_eigvals.unsqueeze(0).repeat(top_eigvecs.shape[0], 1)
PE = torch.cat((eigvals_ep.unsqueeze(1), top_eigvecs.unsqueeze(1)), dim=1)
PE_val = PE.unsqueeze(0).repeat(x_val.shape[0], 1, 1, 1)

In [ ]:
y_val_p = renormalize(model(x_val, PE_val))
relative_error(y_val_t[..., 0], y_val_p[..., 0]), relative_error(y_val_t[..., 1:], y_val_p[..., 1:])